<a href="https://colab.research.google.com/github/wickkiey/TPS-March-2021-Kaggle/blob/main/Tabular_Playground_Series_March_2021_Kaggle_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/data/TPS_March_2021

/content/drive/MyDrive/data/TPS_March_2021


In [16]:
!ls

sample_submission.csv	   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [17]:
%cd ..

/content/drive/My Drive/data/TPS_March_2021


In [21]:
!pip install -qq category_encoders
!pip install catboost

     |████████████████████████████████| 65.7MB 50kB/s 


In [22]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from catboost import CatBoostClassifier, Pool, cv
import category_encoders as ce 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [23]:
sub = pd.read_csv("data/sample_submission.csv")
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [25]:
train.target.value_counts()

0    220539
1     79461
Name: target, dtype: int64

In [26]:
cat_cols = train.select_dtypes('object').columns.to_list()
print("Category Columns ",cat_cols)

Category Columns  ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']


In [27]:
X = train.drop(["id","target"],axis=1)
y = train['target']

In [28]:
cat_idxs = []
for col in cat_cols:
    cat_idxs.append(X.columns.get_loc(col))
    
print("Category Column indexes",cat_idxs)

Category Column indexes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [42]:
mdl = CatBoostClassifier(iterations = 1000,cat_features=cat_idxs,verbose=200,task_type='GPU',auto_class_weights='Balanced')

In [43]:
cv_score = cross_val_score(mdl,X,y,scoring ='roc_auc',cv=3)

Learning rate set to 0.025189
0:	learn: 0.6821000	total: 91.9ms	remaining: 1m 31s
200:	learn: 0.4411137	total: 15.4s	remaining: 1m 1s
400:	learn: 0.4287041	total: 30.7s	remaining: 45.8s
600:	learn: 0.4230236	total: 45.8s	remaining: 30.4s
800:	learn: 0.4191488	total: 1m	remaining: 15.1s
999:	learn: 0.4160640	total: 1m 16s	remaining: 0us
Learning rate set to 0.025189
0:	learn: 0.6821467	total: 72.5ms	remaining: 1m 12s
200:	learn: 0.4397306	total: 15.3s	remaining: 1m
400:	learn: 0.4275539	total: 30.6s	remaining: 45.7s
600:	learn: 0.4218732	total: 45.6s	remaining: 30.3s
800:	learn: 0.4180768	total: 1m	remaining: 15.1s
999:	learn: 0.4152469	total: 1m 15s	remaining: 0us
Learning rate set to 0.025189
0:	learn: 0.6820577	total: 71.2ms	remaining: 1m 11s
200:	learn: 0.4407294	total: 15.2s	remaining: 1m
400:	learn: 0.4285405	total: 30.5s	remaining: 45.5s
600:	learn: 0.4228195	total: 45.6s	remaining: 30.3s
800:	learn: 0.4188904	total: 1m	remaining: 15.1s
999:	learn: 0.4158130	total: 1m 15s	remaini

In [44]:
cv_score

array([0.88904015, 0.88713331, 0.88688019])

In [32]:
mdl.fit(X,y)

Learning rate set to 0.024634
0:	learn: 0.6800398	total: 128ms	remaining: 2m 7s
200:	learn: 0.3705679	total: 20.5s	remaining: 1m 21s
400:	learn: 0.3604383	total: 41s	remaining: 1m 1s
600:	learn: 0.3562573	total: 1m 1s	remaining: 40.7s
800:	learn: 0.3537206	total: 1m 21s	remaining: 20.3s
999:	learn: 0.3517915	total: 1m 41s	remaining: 0us


In [34]:
test_pred = mdl.predict_proba(test[X.columns])

In [39]:
res = pd.DataFrame()
res['id'] = test['id']
res['target'] = test_pred[:,1]
res.to_csv("predict.csv",index=False)

In [37]:
res.head()

,id,target
0,5,0.853213
1,6,0.440921
2,8,0.963542
3,9,0.843102
4,11,0.914338


In [38]:
# from google.colab import files

# files.download('predict.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
!kaggle competitions submit -c tabular-playground-series-mar-2021 -f predict.csv -m "Catboost 2k GPU direct push"

100% 5.06M/5.06M [00:05<00:00, 1.04MB/s]
Successfully submitted to Tabular Playground Series - Mar 2021